In [2]:
import os
from modulus.sym.hydra import to_absolute_path
from csv_rw import csv_to_dict
import csv
import numpy as np
import glob
from multiprocessing import Process
from fwdFacingStep import ffs, param_ranges, Re, Ho, Lo


In [3]:
def readFile(file_path):
    with open(file_path, "r") as datafile:
        data = []
        reader = csv.reader(datafile, delimiter=",")
        for row in reader:
            columns = [row[1]]
            data.append(columns)
        last_row = float(data[-1][0])
        return last_row

In [41]:
ansysFilePath="./ansysDPsValidation.csv"
resultsFilePath="./results.csv"
outputsPath="./outputs/fwdFacingStep"

models = ["old_dataPlusPhysics3600@300k", "data3600PlusPhysicsLambda05@300k", "data3600PlusPhysicsLambda1@300k", "old_physicsOnly@500k", "physicsOnly@500k"]

diffSumSkip = ["DP 5","DP 36","DP 79","DP 86"]


with open(ansysFilePath, "r") as ansysFile,  open(resultsFilePath, "w") as resultsFile:
        reader = csv.reader(ansysFile, delimiter=",")
        writer = csv.writer(resultsFile, delimiter=",")
        diffDict= {}
        for model in models:
                diffDict[model] = 0
                # print(type(diffDict[model]))
        for i, row in enumerate(reader):
                if i == 0:
                        # add columns for difference modulus vs ansys
                        row.insert(5, "Diff Delta Cp [%]")
                        # row.insert(7, "Diff Delta Cptot [%]")
                        writer.writerow(row)
                else:
                        # add ansys results to table
                        row.insert(5, "-")
                        # row.insert(7, "-")
                        writer.writerow(row)
                        for model in models:
                                monitorFilePath = os.path.join(outputsPath, model, "monitors")
                                # read pressure monitors
                                filePattern="downstreamPressure_" + row[0].replace(" ","") + "_*"
                                monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                DSP=readFile(monitorFile[0])
                                # filePattern="downstreamPressureTot_" + row[0].replace(" ","") + "_*"
                                # monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                # DSPtot=readFile(monitorFile[0])
                                filePattern="upstreamPressure_" + row[0].replace(" ","") + "_*"
                                monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                USP=readFile(monitorFile[0])
                                # filePattern="upstreamPressureTot_" + row[0].replace(" ","") + "_*"
                                # monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                # USPtot=readFile(monitorFile[0])
                                
                                # calculate modulus pressure drop and difference modulus vs ansys
                                deltaCp=2*(USP-DSP)
                                # deltaCptot=2*(USPtot-DSPtot)
                                diffDeltaCp=100*(deltaCp-float(row[4]))/float(row[4])
                                # diffDeltaCptot=100*(deltaCptot-float(row[6]))/float(row[6])
                                
                                print(row[0])
                                if row[0] not in diffSumSkip:
                                        print("ns", row[0])
                                        diffDict[model] += abs(diffDeltaCp)
                                
                                # add modulus results to table
                                modRow=[row[0] + " " + model] + row[1:4] + [deltaCp] + [diffDeltaCp] + [DSP] + [USP]
                                # modRow=[row[0] + " " + model] + row[1:4] + [deltaCp] + [diffDeltaCp] + [deltaCptot] + [diffDeltaCptot] + [DSP] + [DSPtot] + [USP] + [USPtot]
                                writer.writerow(modRow)
        # Add deviation sum
        for model in models:
                deviationRow = ["Sum " + model] + ["-"] + ["-"] + ["-"] + ["-"] + [diffDict[model]] + ["-"] + ["-"]
                writer.writerow(deviationRow)
                
                                
                                


DP 5
DP 5
DP 5
DP 5
DP 5
DP 36
DP 36
DP 36
DP 36
DP 36
DP 79
DP 79
DP 79
DP 79
DP 79
DP 86
DP 86
DP 86
DP 86
DP 86
DP 154
ns DP 154
DP 154
ns DP 154
DP 154
ns DP 154
DP 154
ns DP 154
DP 154
ns DP 154
DP 155
ns DP 155
DP 155
ns DP 155
DP 155
ns DP 155
DP 155
ns DP 155
DP 155
ns DP 155
DP 156
ns DP 156
DP 156
ns DP 156
DP 156
ns DP 156
DP 156
ns DP 156
DP 156
ns DP 156
DP 157
ns DP 157
DP 157
ns DP 157
DP 157
ns DP 157
DP 157
ns DP 157
DP 157
ns DP 157
DP 158
ns DP 158
DP 158
ns DP 158
DP 158
ns DP 158
DP 158
ns DP 158
DP 158
ns DP 158
DP 159
ns DP 159
DP 159
ns DP 159
DP 159
ns DP 159
DP 159
ns DP 159
DP 159
ns DP 159
DP 160
ns DP 160
DP 160
ns DP 160
DP 160
ns DP 160
DP 160
ns DP 160
DP 160
ns DP 160
DP 161
ns DP 161
DP 161
ns DP 161
DP 161
ns DP 161
DP 161
ns DP 161
DP 161
ns DP 161
DP 162
ns DP 162
DP 162
ns DP 162
DP 162
ns DP 162
DP 162
ns DP 162
DP 162
ns DP 162
DP 163
ns DP 163
DP 163
ns DP 163
DP 163
ns DP 163
DP 163
ns DP 163
DP 163
ns DP 163
DP 164
ns DP 164
DP 164
ns DP 164
D

In [21]:
import numpy as np

dirSkip = [".hydra", "init"]

optResultsPath = "./optimizationResults/"

models = ["physicsOnly@500k"]
# models = listdir(outputsPath)
models.sort()

for model in models:
    if model in dirSkip:
        print("skipping ", model)
        continue
        
    optPath = optResultsPath + model
    for reNr in range (100, 1100, 100):

        F = np.load(optPath + "/optResultsF" + str(reNr) + ".npy")
        X = np.load(optPath + "/optResultsX" + str(reNr) + ".npy")
        popF = np.load(optPath + "/popF" + str(reNr) + ".npy")
        popX = np.load(optPath + "/popX" + str(reNr) + ".npy") 
        # print("Re " + str(re) + " F: ", F)
        print("Re " + str(reNr) + ": Lo = " + str(X[0]), " Ho = " + str(X[1]))

Re 100: Lo = 0.4992036946570908  Ho = 0.24759940927043964
Re 200: Lo = 0.468799680085605  Ho = 0.24206058221868754
Re 300: Lo = 0.2752192130293616  Ho = 0.4505409573457049
Re 400: Lo = 0.46647198199767737  Ho = 0.3259645207367429
Re 500: Lo = 0.4641116490118472  Ho = 0.3665257016338343
Re 600: Lo = 0.46402240598051586  Ho = 0.35676000416032794
Re 700: Lo = 0.46063345712358583  Ho = 0.3618633578374334
Re 800: Lo = 0.9099537192701446  Ho = 0.3033702229836598
Re 900: Lo = 0.9074608811736081  Ho = 0.3055324061737196
Re 1000: Lo = 0.8306751812605733  Ho = 0.3183161637435419


In [11]:
from os import walk, listdir
i = 1
dir = "./outputs/fwdFacingStep"
print(listdir(dir))
# for root, dirs, files in walk("./outputs/fwdFacingStep"):
#     print(i)
#     print(dirs)
#     i+=i

['pressureDataPlusPhysicsLambda01@500k', 'data3600PlusPhysicsLambda01@500k', 'data1800PlusPhysicsLambda01@300k', 'pressureDataPlusPhysicsLambda05@300k', '.hydra', 'physicsOnly@100k', 'data3600PlusPhysicsLambda01@300k', 'data3600PlusPhysicsLambda05@300k', 'data3600PlusPhysicsLambda1@300k', 'data3600PlusPhysicsLambda05@500k', 'pressureDataPlusPhysicsLambda05@100k', 'data3600PlusPhysicsLambda01@100k', 'pressureDataPlusPhysicsLambda01@100k', 'data3600PlusPhysicsLambda05@100k', 'physicsOnly@300k', 'physicsOnly@500k', 'data3600PlusPhysicsLambda1@100k', 'data1800PlusPhysicsLambda01@500k', 'data3600PlusPhysicsLambda1@500k', 'data1800PlusPhysicsLambda01@100k', 'pressureDataPlusPhysicsLambda01@300k']


In [5]:
t

('hej', 2)